In [4]:
import requests
from bs4 import BeautifulSoup

### 4.6 Step 4
 Извлечение названий товаров с веб-сайта

In [62]:
url = 'http://parsinger.ru/html/index3_page_1.html'

In [63]:
r = requests.get(url)
if r.status_code != 200:
    print(r.status_code, '\n\n', r.text)

In [64]:
r.encoding = 'utf-8'

In [65]:
soup = BeautifulSoup(r.text, 'html.parser')

In [72]:
last_page = int(soup.find('div', {'class': 'pagen'}).find_all('a')[-1].text)

In [73]:
n = 1
full_product = []
for page in range(1, last_page+1):
    url = f'http://parsinger.ru/html/index3_page_{n}.html'
    r = requests.get(url)
    if r.status_code != 200:
        print(r.status_code, '\n\n', r.text)
    r.encoding = 'utf-8'
    soup = BeautifulSoup(r.text, 'html.parser')
    page_product = list()
    for div_teg in soup.find_all('div', {'class': 'img_box'}):
        page_product.append(div_teg.find('a', {'class': 'name_item'}).text)
    full_product.append(page_product)
    n += 1

In [75]:
print(full_product)

[[' Vampire RGB,9 кнопок ', 'Defender Halo Z GM-430L', ' Defender Shark 2 ', ' Defender sTarx GM-390L ', ' Defender Skull GM-180L ', ' Defender Killer GM-170L ', ' Defender Ghost GM-190L ', ' Defender Witcher GM-990 '], [' Logitech G305 LIGHTSPEED Wireless ', ' Logitech G102 LIGHTSYNC Gaming', ' Logitech G PRO HERO Black ', ' Logitech G402 Hyperion Fury ', ' STM 109CW ', ' Defender MB-986 ', ' Gembird MG-520 ', ' CBR CM 846 '], [' Гарнизон GM-720G ', ' Гарнизон GM-700G ', ' Гарнизон GM-760G ', ' Гарнизон GM-730G ', ' Гарнизон GM-620G ', ' A4TECH Bloody Q51 ', ' MGK-03U Dialog Gan-Kata ', ' Defender Flash MB-600L '], [' Gembird MG-550 ', ' Lenovo Legion M200 ', ' Sven RX-G750 ', ' Wired Gaming Mouse ', ' MSI Clutch GM08 ', '  Gembird MG-760 ', ' MGK-11U Dialog Gan-Kata ', ' Logitech G102 ']]


### 4.6 Step 5
 Парсинг артикулов товаров с веб-сайта

In [122]:
url = 'http://parsinger.ru/html/index3_page_4.html'

In [123]:
r = requests.get(url)
r.encoding = 'utf-8'

In [124]:
soup = BeautifulSoup(r.text, 'html.parser')

In [125]:
last_page = int(soup.find('div', class_='pagen').find_all('a', {'href': True})[-1].text)
last_page

4

In [132]:
n = 1
total_sum = 0
for page in range(1, last_page+1): 
    url_product = f'https://parsinger.ru/html/index3_page_{n}.html'
    r_page = requests.get(url_product)
    r_page.encoding = 'utf-8'
    soup_page = BeautifulSoup(r_page.text)
    href_str = soup_page.find_all('a', {'class': 'name_item', 'href': True})
    href_list = [string.get('href') for string in href_str]
    url_ = 'https://parsinger.ru/html/'
    for href in href_list:
        url_product = url_+href
        r_p = requests.get(url_product)
        r_p.encoding = 'utf-8'
        soup_product = BeautifulSoup(r_p.text)
        total_sum += int(soup_product.find('p', {'class': 'article'}).text.split(' ')[1])
    n += 1    

In [133]:
total_sum

2505109532

### 4.6 Step 6
 Комплексное извлечение стоимости товаров

In [5]:
def soup_html(url):
    r = requests.get(url)
    r.encoding = 'urf-8'
    return BeautifulSoup(r.text, 'html.parser')

In [6]:
url = 'http://parsinger.ru/html/index1_page_1.html'

In [10]:
soup_url_index = soup_html(url)
col_indexs = len(soup_url_index.find('div', class_='nav_menu').find_all('a'))  # кол-во индексов (категорий)

In [26]:
url_href = 'http://parsinger.ru/html/'
index = 1
page = 1
col_price = 0
total_sum = 0
for index in range(1, col_indexs+1):    
    """ Проход по категориям """    
    page_soup = soup_html(f'http://parsinger.ru/html/index{index}_page_{page}.html')
    col_pages = len(page_soup.find('div', {'class': 'pagen'}).find_all('a'))  # кол. страниц (пагенаций)
    for page in range(1, col_pages+1):
        """Проход по страницам"""
        cart_soup = soup_html(f'http://parsinger.ru/html/index{index}_page_{page}.html')
        href_strs = cart_soup.find_all('a', {'class': 'name_item', 'href': True, 'name': True})
        href_list = [href.get('href') for href in href_strs]
        for href in href_list:            
            """Проход по карточкам"""
            url_cards = url_href + href            
            soup_card = soup_html(url_cards)
            card_price = int(soup_card.find('span', {'id': 'price'}).text.split(' ')[0])
            card_col = int(soup_card.find('span', {'id': 'in_stock'}).text.split(' ')[-1])
            col_price = card_price * card_col
            total_sum += col_price            

index1
page1
url запроса http://parsinger.ru/html/index1_page_1.html
url href http://parsinger.ru/html/watch/1/1_1.html
card_price 2310, card_col 5, total_sum 11550
url href http://parsinger.ru/html/watch/1/1_2.html
card_price 5480, card_col 15, total_sum 93750
url href http://parsinger.ru/html/watch/1/1_3.html
card_price 21810, card_col 15, total_sum 420900
url href http://parsinger.ru/html/watch/1/1_4.html
card_price 21810, card_col 22, total_sum 900720
url href http://parsinger.ru/html/watch/1/1_5.html
card_price 27770, card_col 10, total_sum 1178420
url href http://parsinger.ru/html/watch/1/1_6.html
card_price 27770, card_col 7, total_sum 1372810
url href http://parsinger.ru/html/watch/1/1_7.html
card_price 24230, card_col 8, total_sum 1566650
url href http://parsinger.ru/html/watch/1/1_8.html
card_price 32600, card_col 18, total_sum 2153450
page2
url запроса http://parsinger.ru/html/index1_page_2.html
url href http://parsinger.ru/html/watch/1/1_9.html
card_price 24230, card_col 40